In [1]:
import numpy as np

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook

import astropy.units as u
from astropy.coordinates import Angle

import asyncio

from lsst.ts.salobj import Remote, State

import SALPY_ATPtg
import SALPY_ATMCS

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
atptg = Remote(SALPY_ATPtg)
atmcs = Remote(SALPY_ATMCS)

In [22]:
await atmcs.cmd_start.start()

CommandIdAck(cmd_id=1559595944, ack=(ack=303, error=0, result='Ok'))

In [ ]:
await atptg.cmd_start.start()

In [ ]:
await atmcs.cmd_enable.start()

In [ ]:
await atptg.cmd_enable.start()

In [20]:
atmcs.evt_summaryState.get()

In [ ]:
# pos = await atmcs.tel_mountEncoders.next(flush=True, timeout=60)
# print(pos.elevationCalculatedAngle)

In [24]:
a = atmcs.tel_mountEncoders.get()

In [25]:
a.azimuthCalculatedAngle

-100.99909990937158

In [13]:
cts = await atptg.tel_currentTargetStatus.next(flush=True)

In [12]:
cts.demandAz

'-77:26:13.61'

In [14]:
class LivePlot:
    def __init__(self):
#         self.fig = plt.figure()
#         self.ax1 = self.fig.add_subplot(111)

#         plt.ion()    
        self.az = np.array([0.])
        self.el = np.array([0.])
        self.demandRot = np.array([0.])
        
        self.mcs_az = np.array([0.])
        self.mcs_el = np.array([0.])        
        
        self.my_figure = figure(plot_width=400, plot_height=400)

        data = {'x': self.mcs_az,
                'y': self.mcs_el}

        self.target_data = ColumnDataSource(data=data)
        self.position_data = ColumnDataSource(data={'x': [],
                                                    'y': []})
        
        self.target_plot = self.my_figure.circle("x", "y", 
                                                 color="blue", 
                                                 fill_alpha=0.2, 
                                                 size=5,
                                                 source=self.target_data)
        self.position_plot = self.my_figure.circle("x", "y", 
                                                   color="red", 
                                                   size=0.5,
                                                   source=self.position_data)
                                                    
        theta = np.linspace(0., 2.*np.pi, 100)
        
        self.circles = []
        
        for radii in [10., 25., 40., 55., 70.]:
            x, y = radii*np.cos(theta), radii*np.sin(theta)
            self.circles.append(self.my_figure.line("x", "y", source={'x': x, 'y': y}))

        self.handle = show(self.my_figure, notebook_handle=True)

    def atptg_callback(self, id_data):
        self.az = np.append(self.az, Angle(id_data.demandAz, unit=u.deg).deg)
        self.el = np.append(self.el, Angle(id_data.demandEl, unit=u.deg).deg)
        self.demandRot = np.append(self.demandRot, Angle(id_data.demandRot, unit=u.deg).deg)

        print(Angle(id_data.demandAz, unit=u.deg).deg,
              Angle(id_data.demandEl, unit=u.deg).deg)
        new_data = {
            'x' : [Angle(id_data.demandAz, unit=u.deg).deg,],
            'y' : [Angle(id_data.demandEl, unit=u.deg).deg,],
        }

        self.test_data.stream(new_data, 300)
        
        push_notebook(handle=self.handle)

    def atmcs_target_callback(self, id_data):
        new_data = {
            'x' : [(90.-id_data.elevation)*np.cos(np.radians(id_data.azimuth)),],
            'y' : [(90.-id_data.elevation)*np.sin(np.radians(id_data.azimuth)),],
        }

        self.target_data.stream(new_data)
        
        push_notebook(handle=self.handle)

    def atmcs_position_callback(self, id_data):
        new_data = {
            'x' : [(90.-id_data.elevationCalculatedAngle)*np.cos(np.radians(id_data.azimuthCalculatedAngle)),],
            'y' : [(90.-id_data.elevationCalculatedAngle)*np.sin(np.radians(id_data.azimuthCalculatedAngle)),],
        }

        self.position_data.stream(new_data)
        
        push_notebook(handle=self.handle)



In [15]:
live_plot = LivePlot()

In [16]:
# atptg.tel_currentTargetStatus.callback = live_plot.atptg_callback
atmcs.evt_target.callback = live_plot.atmcs_target_callback
atmcs.tel_mountEncoders.callback = live_plot.atmcs_position_callback

In [17]:
asyncio.get_event_loop().run_until_complete(asyncio.sleep(10))

In [ ]:
asyncio.get_event_loop().run_forever()

In [ ]:
print(State(atmcs.evt_summaryState.get().summaryState),
      State(atptg.evt_summaryState.get().summaryState))

In [ ]:
import os

In [ ]:
os.environ["LSST_DDS_DOMAIN"]